## 02_WordCount: A First Look at WordCount

We'll use this popular algorithm to learn some idioms for working with data using Spark's [RDD](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD) API, the original API. 

The classic, simple *Word Count* algorithm is easy to understand and it's suitable for parallel computation, so it's a good vehicle when first learning a Big Data API.

In *Word Count*, you read a corpus of documents, tokenize each one into words, and count the occurrences of all the words globally.

The corresponding Spark program is [WordCount2.scala](https://github.com/deanwampler/spark-scala-tutorial/blob/master/src/main/scala/sparktutorial/WordCount2.scala). It shows you how to structure a Spark program, including imports and one way to construct the required [SparkContext](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.SparkContext)

We'll use the KJV Bible text again. Subsequent exercises will add the ability to specify different input sources using command-line arguments.

Let's work through the code. Recall that the notebook environment already created a `SparkContext` instance for us, named `sc`:

In [ ]:
val in  = "../data/kjvdat.txt"    // input file
val out = "output/kjv-wc2"        // output location (directory)

Like the Introduction, we read the text file and convert the lines to lower case.

In [ ]:
val input = sc.textFile("data/kjvdat.txt").map(line => line.toLowerCase)

If you will read `input` several times, cache the data so Spark doesn't reread from disk each time!

input.cache

The following is one long statement. Take the `input` and:
1. Split each line on non-alphanumeric characters (a crude form of tokenization). `flatMap` "flattens" each array returned into a since `RDD` of words
2. Map over each work and return a tuple with the word and a "seed" count of 1.
3. Do the equivalent of a SQL `GROUPBY`, with each word as the key (the first tuple element), then "collapse" the groups by summing the counts, return `(word, N)` tuples.

In [ ]:
val wc = input
  .flatMap(line => line.split("""[^\p{IsAlphabetic}]+"""))
  .map(word => (word, 1))
  .reduceByKey((count1, count2) => count1 + count2)

Why do I have to name each `count*` variable? A popular shorthand is to use the `_` placeholder for each function argument, converting the last line to `reduceByKey(_ + _)`.

Save the results.

> **Note:** If you run the next cell more than once, _delete the output directory first!_ Spark, following Hadoop conventions, won't overwrite an existing directory.

In [ ]:
println(s"Writing output to: $out")
wc.saveAsTextFile(out)

## Recap

We loaded and cache the data, althought caching isn't really useful this time, since we will make a single pass through the data. I left this statement here just to remind you of this feature.

Now we setup a pipeline of operations to perform the word count.

First, the line was split into words using as the separator any run of characters that isn't alphabetic, e.g., digits, whitespace, and punctuation. Note that using the regex `"\\W+"` doesn't work well for non-UTF8 character sets! This also conveniently removes the trailing `~` characters at the end of each line that exist in the file for some reason.

With this regex, `input.flatMap(line => line.split(...))` maps over each line, expanding it into a collection of words, yielding a collection of collections of words. The `flat` part flattens those nested collections into a single, "flat" collection of words.

The next two lines convert the single word "records" into tuples with the word and a count of `1`. In Spark, the first field in a tuple will be used as the default key for joins, group-bys, and the `reduceByKey` we use next.

The `reduceByKey` step effectively groups all the tuples together with the same word (the key) and then "reduces" the values using the passed in function. In this case, the two counts are added together. Hence, we get two-element *records* with unique words and their counts.

Finally, we invoke `saveAsTextFile` to write the final RDD to the output location.

Note that the input and output locations will be relative to the local file system, when running in local mode, and relative to the user's home directory in HDFS (e.g., `/user/$USER`), when a program runs with HDFS (e.g., in Hadoop, Mesos, etc.).

Spark also follows another Hadoop convention for file I/O; the `out` path is actually interpreted as a directory name. It will contain the same `_SUCCESS` and `part-00000` files discussed previously. In a real cluster with lots of data and lots of concurrent tasks, there would be many `part-NNNNN` files.


#### WordCount2 Exercises

At the end of each example source file, you'll find exercises you can try. Solutions for some of them are implemented in the `solns` package. For example, [solns/WordCount2GroupBy.scala](https://github.com/deanwampler/spark-scala-tutorial/blob/master/src/main/scala/sparktutorial/solns/WordCount2GroupBy.scala) solves the "group by" exercise described in `WordCount2.scala`.

## Exercises

### Exercise 1: Understand the output

If you look at the (unsorted) data, you'll find a lot of entries where the word is a number. (Try searching the input text file to find them.) Are there really that many numbers in the bible? If not, where did the numbers come from? Look at the original file for clues. Hint, consider the file format is the following:  `book|chapter#|verse#|verse_text`.

### Exercise 2: Use other versions of the Bible and related texts

The data directory contains similar files for the Tanach (`t3utf.dat` - in Hebrew), the Latin Vulgate (`vuldat.txt`), the Septuagint (`sept.txt` - Greek).
  

### Exercise 3: Sort the output

See the Scaladoc page for [OrderedRDDFunctions](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.OrderedRDDFunctions). Sort the output by word, try both ascending and descending. Note this can be expensive for large data sets! (Why??)

### Exercise 4: Count the number of words ...

Take the output from the previous exercise and count the number of words that start with each letter of the alphabet and each digit. 

### Exercise 5 (Hard): Sort the output by count

You can't use the same approach as in the previous exercise. Hint: See [RDD.keyBy](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD). What's the most frequent word that isn't a "stop word" (too-common words like "the", "he", "it", "a", ...).

### Exercise 6 (Hard): Group the word-count pairs by count

In other words, group together all pairs where the count is 1 (i.e., just one occurrence of those words was found), all pairs where the count is 2, is 3, etc. Sort ascending or descending. Hint: Is there a method for grouping?

### Exercise 7 (Thought Experiment)

Consider the size of each group created in the previous exercise and the distribution of those sizes vs. counts.

What characteristics would you expect for this distribution? That is, which words (or kinds of words) would you expect to occur most frequently? What kind of _distribution_ fits the counts?